In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf
import gradio

In [2]:
model = tf.keras.applications.inception_v3.InceptionV3()

In [12]:
inp = gradio.inputs.ImageUpload(image_width=299, image_height=299, num_channels=3)
out = gradio.outputs.Label(label_names='imagenet1000', max_label_length=None, num_top_classes=8)

iface = gradio.Interface(inputs=inp, 
                         outputs=out,
                         model=model, 
                         model_type='keras')

# iface = gradio.Interface(inputs=inp, 
#                          outputs=out,
#                          model=model, 
#                          model_type='keras')

In [13]:
iface.validate()

In [14]:
iface.launch(inline=True, browser=False, share=True);

NOTE: Gradio is in beta stage, please report all bugs to: a12d@stanford.edu
Model is running locally at: http://localhost:7863/interface.html


Unexpected exception in keepalive ping task
Traceback (most recent call last):
  File "C:\Users\islam\Anaconda3\envs\tensorflow\lib\site-packages\websockets\protocol.py", line 984, in keepalive_ping
    ping_waiter = yield from self.ping()
  File "C:\Users\islam\Anaconda3\envs\tensorflow\lib\site-packages\websockets\protocol.py", line 583, in ping
    yield from self.ensure_open()
  File "C:\Users\islam\Anaconda3\envs\tensorflow\lib\site-packages\websockets\protocol.py", line 658, in ensure_open
    ) from self.transfer_data_exc
websockets.exceptions.ConnectionClosed: WebSocket connection is closed: code = 1001 (going away), no reason


Model available publicly for 8 hours at: http://03244fc8.ngrok.io/interface.html
